In [23]:
target_schema = '<target schema>'
storage_account = '<storage account>'
container = '<container>'
servername = '<server>'
dbname = '<database>'
entity_concurrency = 4

In [24]:
import pyodbc
import struct
from concurrent.futures import ThreadPoolExecutor

sql_token = mssparkutils.credentials.getToken('DW')
token_bytes = sql_token.encode("UTF-16-LE")
token_struct = struct.pack(f'<I{len(token_bytes)}s', len(token_bytes), token_bytes)

SQL_COPT_SS_ACCESS_TOKEN = 1256
conn = pyodbc.connect( 'DRIVER={ODBC Driver 18 for SQL Server};'
                       f'SERVER={servername};'
                       f'DATABASE={dbname}', \
                       attrs_before={SQL_COPT_SS_ACCESS_TOKEN: token_struct})

cursor = conn.execute(  'SELECT DISTINCT [EntityName]'
                        'FROM [DataverseToSql].[BlobsToIngest]'
                        'WHERE [LoadType] = 0 AND [Complete] = 0')

def run_notebook(entity):
    mssparkutils.notebook.run('$$PROCESS_ENTITY_NOTEBOOK_NAME$$', 7200, {
        "entity": entity,
        "target_schema": target_schema,
        "storage_account": storage_account,
        "container": container,
        "servername": servername,
        "dbname": dbname,
        })

with ThreadPoolExecutor(max_workers = entity_concurrency) as executor:
    for result in executor.map(run_notebook, [row[0] for row in cursor]):
        print(result)